# 1 Last week we used the activation function. Why is this not used with backpropagation?

__A:__ The function is not differentiable.

# 2 What is the minimum number of hidden neuron layers needed in order to approximate an arbitrary continuous function, and why?

__A:__ One hidden layer is the minimum. "Universal approximation theorem: Any continuous function
can be approximated by a neural network with a single
hidden layer", slide 54 https://www.uio.no/studier/emner/matnat/ifi/INF3490/h18/timeplan/slides/lecture6-1pp.pdf


Without a hidden layer, it is impossible to approximate non-linear separable problems. The first layer gives lines, and the second layer opens up the dimension to two, so e.g. traingles can be made. Furthermore, a compbination of enough traingles can approximate any shape. See slide 12 https://www.uio.no/studier/emner/matnat/ifi/INF3490/h18/timeplan/slides/lecture6-1pp.pdf

<mark> UNCLEAR <br>
    
    
    
# 3 Why do we use a validation set? Describe how the three different cross-validation methods presented in the lecture slides work, and what their advantages and disadvantages are.
We want the model to generalize well, meaning that it works well on other data than the data the model was estimated on. When fitting the model to the full dataset, the model will typically not fit well for other data. In oredr to create a model that generalizes well, the data is split into a training and a validation set. The model is fit to the training set, and is then evaluated on the validation set. The model performance on the validation set describes how well the model generalizes. <br>

Three crossvalidation methods. <br> 
1) Split into training and validation (and possible test). Works as the previous paragraph. <br>
2) K-fold cross-validation. Split into K number of folds. Each fold performs as validation set once, while at the same time the model is fit to the remaining folds. <br>
3) Bootstrapping. Draw random observations with replacement and use the rest as validation. Repeat.

# 4 Implement the MLP shown below, and train it to correctly perform the XOR function.

In [30]:
import numpy as np
import matplotlib.pyplot as plt
class NN:
    """ 2 layer (1 hidden). Single input combination of 2 intergers. Output 2 integers"""
    
    def __init__(self, nodeNumbers, activationFunction, learningRate, targets, inputs, errorTolerance, \
                 maxIterations, test=False):
        self.nodeNumbers, self.activationFunction, self.learningRate, self.targets, self.inputs, self.errorTolerance,  \
        self.maxIterations, self.test= nodeNumbers, activationFunction, learningRate, targets, inputs, errorTolerance,\
        maxIterations, test
        
        self.weightMatrices = []
        
        if not test:
            for nodeNumber in range(len(nodeNumbers)-1):
                self.weightMatrices.append(np.random.random_sample((nodeNumbers[nodeNumber+1], \
                                                                    nodeNumbers[nodeNumber+1] +1)) - .5)
            print('self.weightMatrices', self.weightMatrices)
            
        else:
            self.weightMatrices.append(np.array(((1., -1., 0.), (1., 0., 1.))))
            self.weightMatrices.append(np.array(((1., 1., 0.), (1., -1., 1.)))) # Assumed same weights bias nodes both layers
            self.inputs = np.array((1, 0, 1))
            print('self.weightMatrices', self.weightMatrices)
            
    def run(self):
        self.forward()
        self.calculateErrors()
        self.iterations = 1
        self.outputForPlot1 = []
        self.outputForPlot2 = []
        while np.abs(self.error2) > self.errorTolerance and self.iterations < self.maxIterations: 
            self.backward()
            self.forward()
            #print('Output: ', self.outputs)
            self.calculateErrors()
            #print('self.error2',self.error2)

            self.iterations += 1
            self.outputForPlot1.append(self.outputs[-1][0])
            self.outputForPlot2.append(self.outputs[-1][1])
        #self.plot()

        print('Iterations: ', self.iterations, '|Error|: ', self.error2, 'Output: ', self.outputs[1])
        
    def plot(self):
        fig, (ax, ax2) = plt.subplots(1,2)
        ax.plot(np.arange(len(self.outputForPlot1)), self.outputForPlot1)#, label='Input 1')
        ax2.plot(np.arange(len(self.outputForPlot2)), self.outputForPlot2)#, label='Input 2')
        ax.set_title('Input 1')
        ax2.set_title('Input 22')
        plt.savefig('simple.pdf')
        
    def differentiateActivationFunction(self, weightedInputs):
        #print('weightedInputs ',weightedInputs)
        if isinstance(weightedInputs, np.ndarray):
            if weightedInputs.ndim == 1:
                onesArray = np.ones(len(weightedInputs))
        else:
            onesArray = 1
        out = self.activationFunction(weightedInputs)*(onesArray - self.activationFunction(weightedInputs))
        #print(self.activationFunction(weightedInputs), onesArray, out)
        #print('out ', out)
        #return 1
        return out
        
        
    
    def forward(self):
        weightedInputs = []
        weightedInputs.append(self.weightMatrices[0] @ self.inputs)
        self.outputs = []
        self.outputs.append(self.activationFunction(weightedInputs[-1]))

        for layer in range(1, len(self.nodeNumbers)-1):
            outputsIncludingBias = np.concatenate([[1], self.outputs[-1]])
            weightedInputs.append(self.weightMatrices[layer] @ outputsIncludingBias)
            self.outputs.append(self.activationFunction(weightedInputs[-1]))
        self.weightedInputs = weightedInputs
        #print('self.outputs', self.outputs[-1])
            
    def calculateErrors(self):
        self.error2 = 0
        for outputNodeNumber in range(len(self.outputs)):
            self.error2 += (self.targets[outputNodeNumber] - \
                           self.outputs[-1][outputNodeNumber])**2
        
    def backward(self):
        # Output deltas
        self.deltaVectors = []
        '''
        deltaValuesOld = np.array([(self.outputs[-1][outputNumber] - self.targets[outputNumber])\
                      *self.differentiateActivationFunction(self.weightedInputs[-1][outputNumber]) \
                      for outputNumber in range(len(self.targets))])
        '''
        deltaValues = (self.outputs[-1] - self.targets)* self.differentiateActivationFunction(self.weightedInputs[-1])
        self.deltaVectors.append(deltaValues)
        
        # Hidden deltas
        for layerNumber in range(len(self.nodeNumbers)-2):
            deltaVecs = []
            for j in range(self.nodeNumbers[-1-(layerNumber+1)+1]):
                deltaSum = 0
                for k in range(len(self.targets)):
                    deltaSum += self.deltaVectors[-1-layerNumber][k] * self.weightMatrices[-1][k,j+1] 
                deltaVecs.append(deltaSum*self.differentiateActivationFunction(self.weightedInputs[-1][j]))
        self.deltaVectors.insert(0, deltaVecs)
        
        # Output weights
        outputsIncludingBias = np.concatenate([[1], self.outputs[-2]])
        for j in range(self.nodeNumbers[-2] + 1):
            for k in range(len(self.targets)):
                self.weightMatrices[-1][k, j] -= \
                self.learningRate*self.deltaVectors[1][k]*outputsIncludingBias[j]
        
        # Hidden weights
        for inputNumber in range(self.nodeNumbers[0]+1):
            for hiddenNodeNumber in range(self.nodeNumbers[1]): # fix indexes
                self.weightMatrices[0][hiddenNodeNumber, inputNumber] -= self.learningRate \
                * self.deltaVectors[0][hiddenNodeNumber]*self.inputs[inputNumber]
     

 

def activationFunction(x):
    return 1./(1+np.exp(-x))

def activationFunction(x):
    return x
 

numberOfNodes,  activationFunction, learningRate, targets, inputs, errorTolerance, maxIterations = \
[2,2,2], activationFunction, 0.1, [1, 0], [0, 1], 1e-6, 10000
test = True
nn=NN(numberOfNodes, activationFunction, learningRate, targets, inputs, errorTolerance, maxIterations, test)
'''
print('nn.weightMatrices',nn.weightMatrices)

nn.forward()
print('nn.weightedInputs',nn.weightedInputs)
print('nn.outputs',nn.outputs)

nn.calculateErrors()
print('nn.error2',nn.error2)

nn.backward()
print('nn.deltaVectors',nn.deltaVectors)
print('weightMatrices', nn.weightMatrices)
'''
nn.run()


self.weightMatrices [array([[ 1., -1.,  0.],
       [ 1.,  0.,  1.]]), array([[ 1.,  1.,  0.],
       [ 1., -1.,  1.]])]
Iterations:  6 |Error|:  nan Output:  [nan nan]


/home/k/.local/lib/python3.6/site-packages/ipykernel_launcher.py:81: RuntimeWarning: overflow encountered in double_scalars
/home/k/.local/lib/python3.6/site-packages/ipykernel_launcher.py:57: RuntimeWarning: overflow encountered in multiply
/home/k/.local/lib/python3.6/site-packages/ipykernel_launcher.py:57: RuntimeWarning: overflow encountered in double_scalars
/home/k/.local/lib/python3.6/site-packages/ipykernel_launcher.py:113: RuntimeWarning: invalid value encountered in double_scalars


In [20]:
def activationFunction(x):
    return x

numberOfNodes,  activationFunction, learningRate, targets, inputs, errorTolerance, maxIterations = \
[2,2,2], activationFunction, 0.1, [1, 0], np.array((1, 0, 1)), 1e-6, 100
test = False


nn2=NN(numberOfNodes, activationFunction, learningRate, targets, inputs, errorTolerance, maxIterations, test)
nn2.run()

self.weightMatrices [array([[-0.08851358,  0.47923393, -0.01478144],
       [ 0.26991866, -0.12081351,  0.10372475]]), array([[ 0.19270413,  0.37050541,  0.16655994],
       [-0.47359476, -0.29878165,  0.08109803]])]
Iterations:  36 |Error|:  7.936851581157607e-07 Output:  [ 9.99576883e-01 -7.84000802e-04]


In [31]:
# change derivative if use this one
def activationFunction(x):
    return 1./(1+np.exp(-x)) 

numberOfNodes,  activationFunction, learningRate, targets, inputs, errorTolerance, maxIterations = \
[2,2,2], activationFunction, 0.1, [1, 0], np.array((1, 0, 1)), 1e-6, 100000
inputs = np.array((1, 0, 1))

nn3=NN(numberOfNodes, activationFunction, learningRate, targets, inputs, errorTolerance, maxIterations, test=False)
nn3.run()

self.weightMatrices [array([[-0.02629585, -0.02179738,  0.16925185],
       [ 0.23853403,  0.37057834,  0.28275949]]), array([[-0.24233045,  0.39173689,  0.09455386],
       [ 0.37015428,  0.12036227,  0.44036432]])]
Iterations:  100000 |Error|:  5.042944549922127e-05 Output:  [0.99498318 0.00502603]
